<a href="https://colab.research.google.com/github/sumithdcosta/Python/blob/master/9.%20Problems/ParsingMutliValuedFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

9. Problems/ParsingMutliValuedFile.ipynb

processing multi values file

1. multivalues.csv is mutivalued file
2. col1 is key index
3. col3 is mutli valued 
4. mutli valued should be joined with | paramter in target file

output should be

- col1,col2,col3,col4
- 1,x1,y1|y2|y3,z1
- 2,x2,y2,z2
- 3,x3,y3,z3

In [4]:
%%writefile multivalues.csv
col1,col2,col3,col4
1,x1,y1,z1
2,x2,y2,z2
1,x1,y2,z1
1,x1,y3,z1
1,x1,y1,z1
3,x3,y3,z3

Writing multivalues.csv


In [0]:
import csv                           # import csv package to read csv format data
from collections import defaultdict  #defaultdict for mutli value dict for values
d2 = defaultdict(set)                # use of set to remove duplicates from dict values
d3 = defaultdict(set)
d4 = defaultdict(set)
with open('multivalues.csv','rt') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',') # csv reader with comma seperator
  line_count = 0                      # set line count to ignore the header row
  for row in csv_reader:              # here 'row' is LIST object
    if line_count == 0:               # to capture the header row
      header = row
      line_count += 1
    else:
      col1, col2, col3, col4 = row    # named tuple to capture value to objects from row list
      d2[col1].add(col2)              # adding values to defaultdict(which removes duplicate)
      d3[col1].add(col3)
      d4[col1].add(col4)
      line_count += 1
  print('Processed %s lines', line_count) # print row processed messages

Processed %s lines 7


In [0]:
line_count = 0                            # line count to print header back to target file
with open('multivalues.out', 'wt') as csv_file:  
  file_writer= csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  if line_count == 0:
    file_writer.writerow(header)          # writing header, header is List 
    line_count += 1
  for key in d2:                          # reading key value from dict whihc is primary key
    row = key, ('|'.join(str(e) for e in d2[key])), \   # making dict values as generator 
               ('|'.join(str(e) for e in sorted(d3[key]))), \  # soting the output and concatinate with | 
               ('|'.join(str(e) for e in d4[key]))
    file_writer.writerow(row)             # writing data to target file

In [0]:
cat multivalues.out

col1,col2,col3,col4
1,x1,y1|y2|y3,z1
2,x2,y2,z2
3,x3,y3,z3


solving problem with pandas:

In [0]:
import csv
import pandas as pd

csv_read = pd.read_csv('multivalues.csv', names = ['col1','col2','col3','col4'], header = 0)
df = pd.DataFrame(csv_read)
df = df.sort_values('col3')
df = df.groupby(['col1','col2','col4'])['col3'].apply(lambda x: '|'.join(x)).reset_index()
df = df[['col1','col2','col3','col4']]

df.to_csv('multivalues_pd.out', mode ='w', index=False,header=True)

In [9]:
cat multivalues_pd.out

col1,col2,col3,col4
1,x1,y1|y1|y2|y3,z1
2,x2,y2,z2
3,x3,y3,z3
